# How To Escape Bates Motel?  <img src='img/Bates-Motel.jpeg' style="float: right; width: 600px;" alt="Drawing">
Vicky Kwan, Jan 2018

## Contents:
- Motivation
- Data Overview
- __The Map__ (K-means clustering)
- __The Compass__ (Nearest Neighbor Retrieval)
- __The Helicopter__ (Matrix factorization models)
- Conclusion

id tag for session motivation
<a id='Motivation'></a>

## Motivation 

Would Marion Crane have inquired into other visitors' reviews (or lack of in this case) before checking in at Norman Bates' motel, she'd probably end up serving 6 months to 3 years for taking her clinet's $40,000 cash payments, and with a messy breakup from Sam Loomis, her boyfriend. One thing was certain; she'd been alive and escaped Bates' delusional killing.

In the age of _Yelp_ and _Google_ and _TripAdvisor_, there's hardly any (legit) businesses that haven't been critiqued by visitors. Businesses either benefit from this form of advertisement or fear for their market shares in industry. Users of these websites review others' ratings and experiences before making decision to try out for themselves. 

In this project, I present multifaceted study of building a recommender system for hotels based on their ratings and reviews on [Tripadvisor](https://www.tripadvisor.com). We will be looking at our _survival_ from Marion's perspective. She will need: a __map__, a __compass__, and (really desperate for mass production and high quality user experience) a __helicopter__.

- A __Map__ of hotels grouped by similar features like this:

In [5]:
from IPython.display import HTML

HTML("img/clusters.html")

- A __Compass__ to direct us in the high dimensional space, to find the hotel that's most similar to our favorite:
<img src='img/KDTree-animation.gif' style="float: center; height: 300px;" alt="Drawing">

(_Source_: [Wikipedia: kd tree](https://en.wikipedia.org/wiki/K-d_tree))

- A __Helicopter__ to bring up the speed and benefit over 1 million visitors:

<img src='img/spiderman_helicopter.png' style="float: center; height: 300px;" alt="Drawing">

id tag for session data
<a id='Data Overview'></a>

## Data Overview 

id tag for session kmeans
<a id='kmeans'></a>

## K Means Clustering using NLP

To sketch a map of hotels based on their ratings instesad of geographic locations, I applied K-means clustering technique on the reviews written by over 1 million TripAdvisor users for more than 7,000 hotels around the world. My motivation is to detect latent structures from the reviews corpus. 

Reviews were arranged in list structure, whose order matches hotel names. We will see the power of NLP (Natural Language Processing) algorithms and NLTK 3.2.5 (Natural Language Toolkit) in studying and making sense of human language. To make consistent and comparable clustering, we will be only looking at reviews written in English by 900k users. 

Using the tf-idf matrix, we can run k-means clustering to better understand the latent structure of the reviews. K-means initializes with a pre-determined number of clusters. In this case, I picked k=5. Each observation is assigned to a cluster (cluster assignment) so as to minimize the within cluster _sum of squares_. Next, the mean of the clustered observations is calculated and used as the new cluster centroid. Then, observations are reassigned to clusters and centroids recalculated in an iterative process until the algorithm reaches convergence.

The output of the key words from each cluster centroid indicates some _fuzzy_ features that are beyond one-worded or one-phrased tags. For instance, when I ran on a random sample of 100 hotels from dataset, I got the following key words that light up for particular hotels.

| Clusters          | tags           
| ------------------|:-------------:
| Cluster 0 words:  | beach, resorts, ocean, view, family, kid, relax, concierge 
| Cluster 1 words:  | station, city, street, metro, taxi, sleep, shuttle, airport      
| Cluster 2 words:  | downtown, new, street, business, coffee, city, terrace, palace   
| Cluster 3 words:  | romantic, Paris, cafe, quietness, decorated, opera, bars, modern   
| Cluster 4 words:  | piazza, fountain, foods, touring, shops, bars, mall   


Two take-aways:

1. The 'tags' are retrieved by TfIdf vectorizer. The vectorizer started with computing the term frequency and inverse document frequency:

    $$\mathrm{tfidf}(t,d,D) = \mathrm{tf}(t,d) \cdot \mathrm{idf}(t, D)$$

    and ended with a matrix of 'key words' that are most relevant to each hotel. My model then calculates the cosine similarity matrix of the hotel key words. These similarity measures later were fed into the k-means model for segregation by similarity; hotels who received many times the same key words, say, 'romantic', were assigned the same group. This segregation is shown in the above __map__; hotel names will display when you move cursor on the dots.
    
2. Key words have a touch of human intervention; what if the _fuzzy_ feature of a hotel cannot be describe by one word in any language? For instance, the model discovered that in Cluster 3, all hotels share this feature of "romantic, Paris, cafe, quietness, decorated, opera, bars, modern".   


A big drawback of k-means is that we have to choose k. How do we really know how many clusters our data should have? A different approach is hierarchical clustering.

Hierarchical clustering is a type of clustering algorithm where we build nested clusters. We represent this hierarchy as a tree or a dendrogram.

This guide covers:

<ul>
<li> tokenizing and stemming each review
<li> transforming the corpus into vector space using [tf-idf](http://en.wikipedia.org/wiki/Tf%E2%80%93idf)
<li> calculating cosine distance between each document as a measure of similarity
<li> clustering the documents using the [k-means algorithm](http://en.wikipedia.org/wiki/K-means_clustering)
<li> using [multidimensional scaling](http://en.wikipedia.org/wiki/Multidimensional_scaling) to reduce dimensionality within the corpus
<li> plotting the clustering output using [matplotlib](http://matplotlib.org/) and [mpld3](http://mpld3.github.io/)
<li> conducting a hierarchical clustering on the corpus using [Ward clustering](http://en.wikipedia.org/wiki/Ward%27s_method)
<li> plotting a Ward dendrogram
<li> topic modeling using [Latent Dirichlet Allocation (LDA)](http://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)
</ul>



## Nearest Neighbor based on Rating structures

<img src='img/universe.jpg' style="float: right; height: 300px;" alt="Drawing">
The main goal is to recommend the most similar hotel to a selected hotel. There are other ways of recommending hotels (content-based k-means clustering, for one, as introduced in another tutorial, [hotel_clustering_using_nlp](hotel_clustering_using_nlp.ipynb)). To __look up__ the most similar hotel in our __universe of hotels__, imagine your favorite hotel __lives__ on Planet A. Now when you travel to a different planet (sorry, __city__), you want to find a hotel that reminds you of your favorite on Planet A. A k-dimensional tree algorithm can help you locate that hotel in almost no time. I'll implement an __Approximate Nearest Neighbors__ search on a __7 dimenional tree__ that has been tailored to our dataset of 10,000 hotels that have been rated by over 1.04 million users on TripAdvisor.

In this tutorial, I'll implement a fast search algorithm for content-based k-Nearest Neighbor model. KNN models were known for their interpretability in the early days of recommender systems. I'll define a way of measuring similarity, i.e. how similar any of the chosen two hotels are, based on their features. This similarity measure helps guide us in the __universe__ of __similar hotels__. The model compares similarity measures among hotels, to decide how __similar__ a pair of hotels are in the rating structures given by travelers. If we would like to know how __similar__ a pair of items are, kNN models would also be a very intuitive way to tell the __degree of similarity__. 

In statistics, the __curse of dimensionality__ has made many problems a lot harder when the dimensions increase. In a 1D space (a straight line on paper), this search problem is essential a query problem; whatever hotel located next to your favorite is the most similar one. In a 2D plane (laying out a map on table), we can draw circles centered at your favorite hotel to search for the next similar one. In a 3D world like ours, there is yet another dimension added to the previous plane, so search time increases as a result.

Hotels in our model will be measured in 7 dimensions: rooms, service, cleanlines, front desk, business service, value, location; each will be given a 0 to 5 scores, where 1 - 5 are travelers' ratings. By using a k-dimensional tree, the next similar hotel will be generated automatically from the training set that we feed in to the algorithm.

## Content:
<ul>
<li>[Data pre-processing](#Data pre-processing)
<li>[Define distance metric](#Define distance metric)
<li>[Train test split](#Train test split)
<li>[Build a KD tree](#Build a KD tree)
<li>[Search for the nearest neighbor on a KD tree](#Search for the nearest neighbor on a KD tree)
<li>[Testing on sample](#Testing on sample)
</ul>